In [10]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())


True
1


In [13]:
# Hugging Face login and private repo setup
import os
from pathlib import Path
from huggingface_hub import HfApi, login, create_repo

# Read token from env or fallback file
HF_TOKEN = os.getenv('HF_TOKEN')
if not HF_TOKEN:
    token_file = Path('/home/ubuntu/cad-llm/notebooks/hf_token.txt')
    if token_file.exists():
        HF_TOKEN = token_file.read_text().strip()

assert HF_TOKEN, 'HF_TOKEN not set. Set env or create notebooks/hf_token.txt'
login(token=HF_TOKEN)

api = HfApi()
user = api.whoami(token=HF_TOKEN).get('name') or api.whoami(token=HF_TOKEN).get('orgs', [{}])[0].get('name')
HF_REPO = os.getenv('HF_REPO', f'{user}/cadgpt-gpt2-train')
create_repo(HF_REPO, private=True, exist_ok=True, token=HF_TOKEN)
print(f'Repo ready: {HF_REPO} (private)')


Repo ready: polaris314/cadgpt-gpt2-train (private)


In [ ]:
# Configure Trainer to push all checkpoints to the private repo
# Assumes you define `training_args = TrainingArguments(...)` later
from transformers import TrainingArguments

assert 'HF_REPO' in globals(), 'Run the HF login/setup cell first.'

PUSH_EVERY_STEPS = int(os.getenv('PUSH_EVERY_STEPS', '200'))  # align with save_steps

def make_args_with_hub(base: TrainingArguments) -> TrainingArguments:
    base.push_to_hub = True
    base.hub_model_id = HF_REPO
    base.hub_private_repo = True
    base.hub_token = HF_TOKEN
    base.save_strategy = 'steps'
    base.save_steps = max(getattr(base, 'save_steps', 200) or 200, 50)
    base.save_total_limit = 3  # Keep only last 3 checkpoints locally
    base.load_best_model_at_end = True  # Load best model for final push
    base.push_to_hub_model_id = HF_REPO
    base.hub_always_push = False  # Only push final model, not every checkpoint
    base.logging_steps = min(getattr(base, 'logging_steps', 50) or 50, base.save_steps)
    return base

def upload_checkpoint_to_hub(checkpoint_path, step_num):
    """Upload a specific checkpoint to HF Hub with unique naming"""
    from huggingface_hub import HfApi
    import shutil
    import tempfile
    
    api = HfApi(token=HF_TOKEN)
    
    # Create a temporary directory for the checkpoint
    with tempfile.TemporaryDirectory() as temp_dir:
        # Copy checkpoint files to temp directory
        checkpoint_name = f"checkpoint-{step_num}"
        temp_checkpoint_path = os.path.join(temp_dir, checkpoint_name)
        shutil.copytree(checkpoint_path, temp_checkpoint_path)
        
        # Upload to HF Hub with unique name
        repo_id = f"{HF_REPO}-checkpoint-{step_num}"
        api.create_repo(repo_id, private=True, exist_ok=True, token=HF_TOKEN)
        api.upload_folder(
            folder_path=temp_checkpoint_path,
            repo_id=repo_id,
            token=HF_TOKEN
        )
        print(f"✅ Uploaded checkpoint-{step_num} to {repo_id}")

print('Defined make_args_with_hub() and upload_checkpoint_to_hub().')
print('After creating training_args, run:')
print('    training_args = make_args_with_hub(training_args)')


Defined make_args_with_hub(). After creating training_args, run:
    training_args = make_args_with_hub(training_args)


In [15]:
!nvidia-smi

Fri Sep  5 11:18:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 6000                On  |   00000000:07:00.0 Off |                  Off |
| 33%   31C    P8             15W /  260W |       4MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# **air motor**

# **generate dataset**

In [17]:

import pandas as pd, json

# Path to your uploaded Excel in Colab
in_path = "/home/ubuntu/cad-llm/datasets/air_motor.xlsx"
out_stem = "/home/ubuntu/cad-llm/notebooks/air_motor_prompts"

# --- required columns ---
REQUIRED_COLS = [
    "pressure_bar","pressure_mpa","stroke_length",
    "cyl_id","cyl_len","cyl_thk","cyl_od",
    "disc_dia","disc_thk","thru_hole","counterbore","groove_dia","groove_height",
    "head_dia","head_length","neck_dia","neck_length","chamf_dist",
    "piston_dia","piston_length","ext_length_A","ext_dia_A","threaded_depth",
    "flange_dia","flange_thk","hub_od","hub_id","hub_length",
    "ext_dia_B","ext_length_B","center_hole_dia","center_hole_depth",
    "bolt_hole_radius","pattern_radius","small_radius","n_bolts",
]

# --- load file ---
df = pd.read_excel(in_path)

missing = [c for c in REQUIRED_COLS if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

# --- prompt template ---
import random

PROMPT_TEMPLATES = {
    "piston_disc": [
        "Generate a piston disc for an air motor with cylinder ID {cyl_id}mm, stroke length {stroke_length}mm, designed for pressure {pressure_bar} bar.",
        "Design a piston disc for a cylinder of ID {cyl_id} mm and stroke {stroke_length} mm, operating at {pressure_bar} bar.",
        "Create a piston disc model for an air motor (cyl ID {cyl_id} mm, stroke {stroke_length} mm, pressure {pressure_bar} bar).",
        "Build the piston disc of an air motor: bore {cyl_id} mm, stroke {stroke_length} mm, working pressure {pressure_bar} bar."
    ],
    "piston_rod": [
        "Generate a piston rod for an air motor with cylinder ID {cyl_id} mm, stroke {stroke_length} mm, and pressure {pressure_bar} bar.",
        "Design a piston rod for a cylinder (bore {cyl_id} mm, stroke {stroke_length} mm, pressure {pressure_bar} bar).",
        "Create a piston rod model: cyl ID {cyl_id} mm, stroke length {stroke_length} mm, pressure {pressure_bar} bar.",
        "Build the piston rod for an air motor bore {cyl_id} mm, stroke {stroke_length} mm, at {pressure_bar} bar."
    ],
    "flange": [
        "Generate a flange for an air motor with cylinder ID {cyl_id} mm, stroke {stroke_length} mm, and pressure {pressure_bar} bar.",
        "Design a mounting flange for a cylinder (ID {cyl_id} mm, stroke {stroke_length} mm, pressure {pressure_bar} bar).",
        "Create a flange for an air motor: bore {cyl_id} mm, stroke {stroke_length} mm, working pressure {pressure_bar} bar.",
        "Build a flange component for cylinder ID {cyl_id} mm, stroke {stroke_length} mm, operating at {pressure_bar} bar."
    ]
}

def prompt_for(part: str, row: pd.Series) -> str:
    tpl = random.choice(PROMPT_TEMPLATES[part])
    return tpl.format(
        cyl_id=int(round(float(row["cyl_id"]))),
        stroke_length=int(round(float(row["stroke_length"]))),
        pressure_bar=float(row["pressure_bar"])
    )


# --- cadquery code templates ---
CQ_TEMPLATES = {
    "piston_disc": r"""
# piston disc
cbore_height = 3
piston_disc = (cq.Workplane("XY").circle(disc_dia/2).extrude(disc_thk))
piston_disc = (piston_disc.faces(">Z").workplane()
               .cboreHole(thru_hole, counterbore, cbore_height, depth=None))
piston_disc = (piston_disc.faces(">Z").workplane(offset = -(disc_thk - groove_height)/2)
               .circle(groove_dia/2).circle(disc_dia/2).cutBlind(-groove_height))
""",
    "piston_rod": r"""
# piston rod
head = (cq.Workplane("XY").circle(head_dia/2).extrude(head_length).edges("<Z").chamfer(2.5))
neck = (head.faces(">Z").circle(neck_dia/2).extrude(neck_length))
shoulder = (neck.faces(">Z").circle((neck_dia + 5)/2)
            .workplane(offset=chamf_dist).circle(piston_dia/2).loft(combine="a"))
piston = (shoulder.faces(">Z").circle(piston_dia/2).extrude(piston_length))
piston = (piston.faces(">Z").workplane().circle(ext_dia_A/2).extrude(ext_length_A).fillet(0.5))
""",
    "flange": r"""
# flange
flange = (cq.Workplane("XY").circle(flange_dia/2).extrude(flange_thk).edges(">Z").fillet(1))
flange = (flange.faces(">Z").workplane(centerOption="CenterOfMass")
          .polarArray(pattern_radius, 0, 360, int(n_bolts))
          .circle(small_radius).circle(bolt_hole_radius)
          .extrude(-flange_thk).edges("|Z").fillet(5))
flange = (flange.faces(">Z").workplane().hole(center_hole_dia, center_hole_depth))
flange_hub = (flange.faces(">Z").workplane().center(0,0)
              .circle(hub_od/2).circle(hub_id/2).extrude(hub_length))
flange = (flange_hub.faces(">Z").workplane()
          .circle(ext_dia_B/2).circle(hub_id/2).extrude(ext_length_B))

""",
}

def code_for(part, row):
    header_lines = ["import cadquery as cq", "", "# Parameters from dataset row"]
    for k, v in row.items():
        if pd.isna(v): continue
        try:
            fv = float(v)
            header_lines.append(f"{k} = {int(round(fv))}" if abs(fv-round(fv)) < 1e-9 else f"{k} = {fv}")
        except Exception:
            header_lines.append(f"{k} = {repr(v)}")
    return "\n".join(header_lines) + "\n" + CQ_TEMPLATES[part]

# --- generate dataset ---
rows_out = []
for idx, row in df.iterrows():
    for part in ["piston_disc","piston_rod","flange"]:
        rows_out.append({
            "row_id": int(idx),
            "part": part,
            "prompt": prompt_for(part,row),
            "completion": code_for(part,row),
        })

out_csv   = out_stem + ".csv"
out_jsonl = out_stem + ".jsonl"

pd.DataFrame(rows_out).to_csv(out_csv, index=False)
with open(out_jsonl,"w",encoding="utf-8") as f:
    for rec in rows_out:
        f.write(json.dumps(rec, ensure_ascii=False)+"\n")

print(f"✅ Saved {out_csv} and {out_jsonl} with {len(rows_out)} samples "
      f"(3 per input row).")


✅ Saved /home/ubuntu/cad-llm/notebooks/air_motor_prompts.csv and /home/ubuntu/cad-llm/notebooks/air_motor_prompts.jsonl with 30000 samples (3 per input row).


# **train gpt2 on generated data**

In [18]:
# Read Hugging Face Token from local file
try:
    with open('hf_token.txt', 'r') as f:
        HF_TOKEN = f.read().strip()
    print("✅ Hugging Face token loaded successfully")
except FileNotFoundError:
    print("❌ hf_token.txt file not found. Please create it with your HF token.")
    HF_TOKEN = None
except Exception as e:
    print(f"❌ Error reading token file: {e}")
    HF_TOKEN = None

if not HF_TOKEN:
    print("⚠️ Warning: Hugging Face token not available. Some operations may fail.")

✅ Hugging Face token loaded successfully


In [20]:
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)

# CONFIG
DATA_JSON = "/home/ubuntu/cad-llm/notebooks/air_motor_prompts.jsonl"  # adjust path if needed
BASE_MODEL = "gpt2"
OUT_DIR = "/home/ubuntu/cad-llm/cad_llm"
EPOCHS = 2              # start with 2, bump to 3 if eval loss still dropping
BATCH = 2               # per-device batch
GRAD_ACCUM = 4          # effective batch = BATCH * GRAD_ACCUM
MAX_LENGTH = 1024       # GPT-2 context size
LR = 5e-5
SEED = 42


def make_text(example):
    sep = "\n### CADQUERY CODE\n"
    prompt = (example.get("prompt") or "").strip()
    code = (example.get("completion") or "").strip()  # <-- FIXED
    return {"text": prompt + sep + code + "\n"}

def main():
    # load json dataset
    raw_ds = load_dataset("json", data_files=DATA_JSON, split="train")
    # tiny eval split (2%)
    splits = raw_ds.train_test_split(test_size=0.02, seed=SEED)
    train_ds, eval_ds = splits["train"], splits["test"]

    # combine fields -> training text
    train_ds = train_ds.map(make_text, remove_columns=train_ds.column_names)
    eval_ds  = eval_ds.map(make_text, remove_columns=eval_ds.column_names)

    # Pass the token explicitly when loading tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True, token=HF_TOKEN)
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})

    model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, token=HF_TOKEN)
    model.resize_token_embeddings(len(tokenizer))  # <-- align with tokenizer

    def tokenize_fn(batch):
        return tokenizer(
            batch["text"],
            truncation=True,
            max_length=MAX_LENGTH,
            padding="max_length",
            return_attention_mask=True,
        )

    train_tok = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
    eval_tok  = eval_ds.map(tokenize_fn, batched=True, remove_columns=["text"])

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="checkpoints",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH,
        gradient_accumulation_steps=GRAD_ACCUM,   # effective batch > 8k tokens/step
        learning_rate=LR,
        warmup_steps=500,
        weight_decay=0.01,
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="steps",
        save_steps=200,
        save_total_limit=None,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True,   # safe for Colab T4
        report_to="none",
        seed=SEED,
        # Pass the token to the Trainer arguments as well (optional but good practice)
        hub_token=HF_TOKEN,
    )
    
    training_args = make_args_with_hub(training_args)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tok,
        eval_dataset=eval_tok,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )

    trainer.train()
    trainer.save_model(OUT_DIR)
    tokenizer.save_pretrained(OUT_DIR)
    print("✅ Saved finetuned model to", OUT_DIR)

if __name__ == "__main__":
    main()

Map: 100%|██████████| 600/600 [00:00<00:00, 1937.06 examples/s]
/tmp/ipykernel_16922/2436310189.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss,Validation Loss
200,0.298700,0.181584
400,0.123600,0.099739
600,0.087600,0.069223
800,0.070300,0.055413
1000,0.060100,0.053136
1200,0.055300,0.045289
1400,0.051400,0.042706
1600,0.047300,0.040301
1800,0.046300,0.040413
2000,0.046100,0.038608


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            





Processing Files (1 / 2)                :   3%|▎         | 16.8MB /  498MB,   ???B/s  


Processing Files (1 / 2)                :  12%|█▏        | 58.7MB /  498MB,  209MB/s  


Processing Files (1 / 2)                :  20%|██        |  101MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  29%|██▊       |  143MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  37%|███▋      |  184MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  45%|████▌     |  226MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  54%|█████▍    |  268MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  62%|██████▏   |  310MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  69%|██████▉   |  344MB /  498MB,  204MB/s  


Processing Files (1 / 2)                :  79%|███████▉  |  394MB /  498MB,  210MB/s  


Processing Files (1 / 

✅ Saved finetuned model to /home/ubuntu/cad-llm/cad_llm


In [29]:
# Upload model and checkpoints to single repository with organized structure
import shutil
import tempfile
from pathlib import Path

def upload_to_single_repo():
    """Upload final model and important checkpoints to single HF repo"""
    HF_REPO = "polaris314/cadgpt-gpt2-train"
    print(f"🎯 Uploading to single repository: {HF_REPO}")
    
    api = HfApi(token=HF_TOKEN)
    
    # Create repository if it doesn't exist
    create_repo(HF_REPO, private=True, exist_ok=True, token=HF_TOKEN)
    
    # 1. Upload final model to root of repo
    print("🚀 Uploading final model...")
    if os.path.exists("/home/ubuntu/cad-llm/cad_llm"):
        api.upload_folder(
            folder_path="/home/ubuntu/cad-llm/cad_llm",
            repo_id=HF_REPO,
            token=HF_TOKEN,
            commit_message="Upload final trained CAD-LLM model"
        )
        print(f"✅ Final model uploaded to {HF_REPO}")
    else:
        print("❌ Final model directory not found")
    
    # 2. Upload important checkpoints to organized folders
    checkpoints_dir = "/home/ubuntu/cad-llm/notebooks/checkpoints"
    if os.path.exists(checkpoints_dir):
        checkpoint_dirs = [d for d in os.listdir(checkpoints_dir) if d.startswith("checkpoint-")]
        checkpoint_dirs.sort(key=lambda x: int(x.split("-")[1]))
        
        print(f"📦 Found {len(checkpoint_dirs)} total checkpoints...")
        
        # Select important checkpoints: last 3 + mid-training
        total_checkpoints = len(checkpoint_dirs)
        important_checkpoints = []
        
        if total_checkpoints >= 1:
            important_checkpoints.append(checkpoint_dirs[-1])
        if total_checkpoints >= 2:
            important_checkpoints.append(checkpoint_dirs[-2])
        if total_checkpoints >= 3:
            important_checkpoints.append(checkpoint_dirs[-3])
        if total_checkpoints >= 5:
            mid_index = total_checkpoints // 2
            important_checkpoints.append(checkpoint_dirs[mid_index])
        
        print(f"🎯 Uploading {len(important_checkpoints)} important checkpoints...")
        
        # Upload each checkpoint to a subfolder in the same repo
        for checkpoint_dir in important_checkpoints:
            checkpoint_path = os.path.join(checkpoints_dir, checkpoint_dir)
            step_num = checkpoint_dir.split("-")[1]
            
            # Create temp directory with organized structure
            with tempfile.TemporaryDirectory() as temp_dir:
                # Copy checkpoint to temp directory with organized name
                organized_name = f"checkpoint-step-{step_num}"
                temp_checkpoint_path = os.path.join(temp_dir, organized_name)
                shutil.copytree(checkpoint_path, temp_checkpoint_path)
                
                try:
                    # Upload to subfolder in the same repo
                    api.upload_folder(
                        folder_path=temp_checkpoint_path,
                        repo_id=HF_REPO,
                        path_in_repo=f"checkpoints/{organized_name}",
                        token=HF_TOKEN,
                        commit_message=f"Add checkpoint at step {step_num}"
                    )
                    print(f"✅ Checkpoint-{step_num} uploaded to {HF_REPO}/checkpoints/{organized_name}")
                except Exception as e:
                    print(f"❌ Failed to upload checkpoint-{step_num}: {e}")
    else:
        print("❌ Checkpoints directory not found")
    
    print("\n🎉 Upload complete!")
    print(f"🔗 View your model at: https://huggingface.co/{HF_REPO}")
    print("📁 Repository structure:")
    print("  ├── config.json, model.safetensors, etc. (final model)")
    print("  └── checkpoints/")
    print("      ├── checkpoint-step-400/")
    print("      ├── checkpoint-step-600/")
    print("      └── ...")

# Run the upload
upload_to_single_repo()


🎯 Uploading to single repository: polaris314/cadgpt-gpt2-train
🚀 Uploading final model...


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            





Processing Files (1 / 2)                :   3%|▎         | 16.8MB /  498MB,   ???B/s  


Processing Files (1 / 2)                :  12%|█▏        | 58.7MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  20%|██        |  101MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  30%|███       |  151MB /  498MB,  224MB/s  


Processing Files (1 / 2)                :  37%|███▋      |  184MB /  498MB,  210MB/s  


Processing Files (1 / 2)                :  47%|████▋     |  235MB /  498MB,  218MB/s  


Processing Files (1 / 2)                :  56%|█████▌    |  277MB /  498MB,  217MB/s  


Processing Files (1 / 2)                :  67%|██████▋   |  335MB /  498MB,  228MB/s  


Processing Files (1 / 2)                :  74%|███████▍  |  369MB /  498MB,  220MB/s  


Processing Files (1 / 2)                :  83%|████████▎ |  411MB /  498MB,  219MB/s  


Processing Files (1 / 

✅ Final model uploaded to polaris314/cadgpt-gpt2-train
📦 Found 37 total checkpoints...
🎯 Uploading 4 important checkpoints...


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            









































Processing Files (4 / 6)                :   6%|▌         | 92.2MB / 1.49GB,   ???B/s  




















Processing Files (4 / 6)                :  12%|█▏        |  176MB / 1.49GB,  419MB/s  




















Processing Files (4 / 6)                :  17%|█▋        |  260MB / 1.49GB,  420MB/s  




















Processing Files (4 / 6)                :  23%|██▎       |  344MB / 1.49GB,  419MB/s  




















Processing Files (4 / 6)                :  29%|██▊       |  428MB / 1.49GB,  420MB/s  




















Processing Files (4 / 6)                :  34%|███▍      |  512MB / 1.49GB,  420MB/s  




















Processing Files (4 / 6)                :  39%|███▉      |  587MB / 1.49GB,  413MB/s  




















Processing Files (4 / 6)                :  45%|████▍     |  671MB / 1.49GB,  414MB/s  




















Processing Files (4 

✅ Checkpoint-7350 uploaded to polaris314/cadgpt-gpt2-train/checkpoints/checkpoint-step-7350


...

✅ Checkpoint-7200 uploaded to polaris314/cadgpt-gpt2-train/checkpoints/checkpoint-step-7200


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            









































Processing Files (4 / 6)                :   7%|▋         |  101MB / 1.49GB,   ???B/s  




















Processing Files (4 / 6)                :  11%|█         |  168MB / 1.49GB,  336MB/s  




















Processing Files (4 / 6)                :  17%|█▋        |  260MB / 1.49GB,  398MB/s  




















Processing Files (4 / 6)                :  24%|██▎       |  352MB / 1.49GB,  419MB/s  




















Processing Files (4 / 6)                :  30%|██▉       |  444MB / 1.49GB,  430MB/s  




















Processing Files (4 / 6)                :  35%|███▍      |  520MB / 1.49GB,  420MB/s  




















Processing Files (4 / 6)                :  41%|████      |  612MB / 1.49GB,  426MB/s  




















Processing Files (4 / 6)                :  47%|████▋     |  696MB / 1.49GB,  426MB/s  




















Processing Files (4 

✅ Checkpoint-7000 uploaded to polaris314/cadgpt-gpt2-train/checkpoints/checkpoint-step-7000


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            









































Processing Files (4 / 6)                :   6%|▌         | 92.3MB / 1.49GB,   ???B/s  




















Processing Files (4 / 6)                :  12%|█▏        |  185MB / 1.49GB,  460MB/s  




















Processing Files (4 / 6)                :  18%|█▊        |  268MB / 1.49GB,  440MB/s  




















Processing Files (4 / 6)                :  23%|██▎       |  344MB / 1.49GB,  419MB/s  




















Processing Files (4 / 6)                :  29%|██▊       |  428MB / 1.49GB,  419MB/s  




















Processing Files (4 / 6)                :  35%|███▍      |  520MB / 1.49GB,  427MB/s  




















Processing Files (4 / 6)                :  40%|████      |  604MB / 1.49GB,  426MB/s  




















Processing Files (4 / 6)                :  46%|████▌     |  688MB / 1.49GB,  425MB/s  




















Processing Files (4 

✅ Checkpoint-3800 uploaded to polaris314/cadgpt-gpt2-train/checkpoints/checkpoint-step-3800

🎉 Upload complete!
🔗 View your model at: https://huggingface.co/polaris314/cadgpt-gpt2-train
📁 Repository structure:
  ├── config.json, model.safetensors, etc. (final model)
  └── checkpoints/
      ├── checkpoint-step-400/
      ├── checkpoint-step-600/
      └── ...


test code generation using test_prompt

In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_DIR = "/home/ubuntu/cad-llm/cad_llm"  # your saved folder
tok = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_DIR).to("cuda")

def generate_code(prompt, max_new_tokens=600, temperature=0.2, top_p=0.95):
    prefix = prompt.strip() + "\n### CADQUERY CODE\n"
    inps = tok(prefix, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inps,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tok.eos_token_id,
            eos_token_id=tok.eos_token_id,
        )
    text = tok.decode(out[0], skip_special_tokens=True)
    # return only the code after the separator
    return text.split("### CADQUERY CODE", 1)[-1].strip()

test_prompt = "Generate a flange for an air motor with cylinder ID 258 mm, stroke length 123 mm, designed for pressure 6.2 bar."
print(generate_code(test_prompt))


import cadquery as cq

# Parameters from dataset row
pressure_bar = 6.2
pressure_mpa = 0.62
stroke_length = 123
cyl_id = 258
cyl_len = 160.5
cyl_thk = 6
cyl_od = 270
disc_dia = 257
disc_thk = 24.5
thru_hole = 20
counterbore = 1.75
groove_dia = 246
groove_height = 7.5
head_dia = 34.5
head_length = 11
neck_dia = 24.5
neck_length = 13
chamf_dist = 10.1
piston_dia = 41
piston_length = 172.5
ext_length_A = 33
ext_dia_A = 20.5
threaded_depth = 13.7
flange_dia = 275
flange_thk = 19
hub_od = 255
hub_id = 230
hub_length = 4
ext_dia_B = 258
ext_length_B = 4
center_hole_dia = 48
center_hole_depth = 22
bolt_hole_radius = 15.5
pattern_radius = 130
small_radius = 8
n_bolts = 4

# flange
flange = (cq.Workplane("XY").circle(flange_dia/2).extrude(flange_thk).edges(">Z").fillet(1))
flange = (flange.faces(">Z").workplane(centerOption="CenterOfMass")
          .polarArray(pattern_radius, 0, 360, int(n_bolts))
          .circle(small_radius).circle(bolt_hole_radius)
          .extrude(-flange_thk).edges("|

In [40]:
# Improved code generation with better completion handling
def generate_code_improved(prompt: str, tokenizer, model):
    """Generate CadQuery code with better completion handling."""
    prompt_text = prompt.strip() + "\n### CADQUERY CODE\n"
    
    try:
        inputs = tokenizer(
            prompt_text, 
            return_tensors="pt", 
            truncation=True, 
            max_length=MAX_INPUT_LEN,
            padding=True
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
    except Exception as e:
        raise RuntimeError(f"Tokenization failed: {str(e)}")

    with torch.no_grad():
        try:
            gen = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                num_beams=4,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                repetition_penalty=1.1,
                early_stopping=True
            )
        except Exception as e:
            raise RuntimeError(f"Model generation failed: {str(e)}")

    if gen is None or len(gen) == 0:
        raise RuntimeError("Model returned empty generation")

    try:
        # Decode only the new tokens (remove input tokens)
        input_length = inputs['input_ids'].shape[1]
        output_seq = gen[0][input_length:]
        out = tokenizer.decode(output_seq, skip_special_tokens=True)
    except Exception as e:
        raise RuntimeError(f"Decoding failed: {str(e)}")

    if not out.strip():
        raise RuntimeError("Decoded output is empty")

    # Extract code after the separator
    if "### CADQUERY CODE" in out:
        parts = out.split("### CADQUERY CODE", 1)
        code = parts[1].strip() if len(parts) > 1 else out
    else:
        # Look for CadQuery import or model assignment
        m = re.search(r"(import\s+cadquery\b.*)", out, re.S | re.I)
        code = m.group(1).strip() if m else out

    if not code.strip():
        raise RuntimeError("No code extracted from generation")
    
    # Try to complete incomplete code
    code = complete_incomplete_code(code)
    
    return code

def complete_incomplete_code(code: str) -> str:
    """Try to complete incomplete generated code."""
    # Remove incomplete lines at the end
    lines = code.splitlines()
    cleaned_lines = []
    
    for i, line in enumerate(lines):
        # Skip lines that end with incomplete operators
        if line.strip().endswith(('.', '(', '=', '+', '-', '*', '/')):
            # Check if this is the last line or if next line is empty
            if i == len(lines) - 1 or (i < len(lines) - 1 and not lines[i + 1].strip()):
                continue
        cleaned_lines.append(line)
    
    code = "\n".join(cleaned_lines)
    
    # Find the main object variable name
    main_object = None
    for line in lines:
        if re.search(r"(piston_disc|cylinder|model|part|result)\s*=", line):
            match = re.search(r"(\w+)\s*=", line)
            if match:
                main_object = match.group(1)
                break
    
    # Add final assignment if missing
    if main_object and not re.search(r"\b(model|result|part)\s*=", code):
        code += f"\nmodel = {main_object}"
    
    return code

# Test the improved version
print("🔧 Testing improved code generation...")
prompt = "generate a piston disc for a pneumatic cylinder of ID 80 mm, stroke length 120 mm, and pressure of 6 bar"

try:
    # Load model
    tokenizer, model = load_model(MODEL_DIR)
    
    # Generate code with improved function
    raw_code = generate_code_improved(prompt, tokenizer, model)
    
    print("\n--- Improved generated code ---")
    print(raw_code)
    
    # Clean and execute
    code = clean_generated_code(raw_code)
    print("\n--- Cleaned code ---")
    print(code)
    
    # Sanity check
    simple_sanity_check(code)
    
    # Execute and export
    out_file = exec_and_export(code)
    print(f"\n✅ SUCCESS! STEP file created: {out_file}")
    
except Exception as e:
    print(f"❌ Failed: {e}")


🔧 Testing improved code generation...
Loading model from /home/ubuntu/cad-llm/cad_llm...


✅ Model loaded successfully

--- Improved generated code ---
import cadquery as cq

# Parameters from dataset row
pressure_bar = 6.5
pressure_mpa = 0.65
stroke_length = 120
cyl_id = 80
cyl_len = 150
cyl_thk = 2
cyl_od = 84
disc_dia = 79.5
disc_thk = 14
thru_hole = 16
counterbore = 3
groove_dia = 70
groove_height = 7
head_dia = 20
head_length = 7
neck_dia = 10
neck_length = 8
chamf_dist = 8.4
piston_dia = 20
piston_length = 168.5
ext_length_A = 25
ext_dia_A = 16
threaded_depth = 22.7
flange_dia = 96
flange_thk = 14
hub_od = 76
hub_id = 66
hub_length = 4
ext_dia_B = 80
ext_length_B = 4
center_hole_dia = 24
center_hole_depth = 9
bolt_hole_radius = 3
pattern_radius = 48
small_radius = 8
n_bolts = 3

# piston disc
cbore_height = 3
piston_disc = (cq.Workplane("XY").circle(disc_dia/2).extrude(disc_thk))
piston_disc = (piston_disc.faces(">Z").workplane()
               .cboreHole(thru_hole, counterbore, cbore_height, depth=None))
piston_disc = (piston_disc.faces(">Z").workplane(offset = -(disc

In [36]:
# Fix the safe builtins issue
def get_safe_builtins_fixed():
    """Return a restricted set of Python builtins (safe for CadQuery)."""
    allowed = {
        "abs", "min", "max", "round", "sum", "all", "any",
        "int", "float", "len", "range", "enumerate", "zip",
        "str", "bool", "list", "tuple", "dict", "set",
        "__import__"  # Add this back for imports to work
    }
    return {k: getattr(builtins, k) for k in allowed}

# Test with fixed builtins
print("🔧 Testing with fixed builtins...")

try:
    # Load model
    tokenizer, model = load_model(MODEL_DIR)
    
    # Generate code
    raw_code = generate_code_improved(prompt, tokenizer, model)
    
    # Clean code
    code = clean_generated_code(raw_code)
    print("\n--- Code to execute ---")
    print(code)
    
    # Execute with fixed builtins
    safe_globals = {"cq": cq, "__builtins__": get_safe_builtins_fixed()}
    safe_locals = {}
    
    print("\n⚙️ Executing code...")
    exec(code, safe_globals, safe_locals)
    
    # Find the CAD object
    cad_obj = None
    for name in ("model", "result", "part"):
        if name in safe_locals:
            cad_obj = safe_locals[name]
            break
        if name in safe_globals:
            cad_obj = safe_globals[name]
            break
    
    if cad_obj is None:
        print("❌ No CAD object found")
    else:
        print("✅ CAD object found, exporting...")
        out_file = "generated_from_llm.step"
        cq.exporters.export(cad_obj, out_file)
        print(f"✅ SUCCESS! STEP file created: {out_file}")
        print(f"📁 File size: {os.path.getsize(out_file)} bytes")
        
except Exception as e:
    print(f"❌ Failed: {e}")
    import traceback
    traceback.print_exc()


🔧 Testing with fixed builtins...
Loading model from /home/ubuntu/cad-llm/cad_llm...
✅ Model loaded successfully

--- Code to execute ---
import cadquery as cq

# Parameters from dataset row
pressure_bar = 6.5
pressure_mpa = 0.65
stroke_length = 120
cyl_id =  80
cyl_len =  150
cyl_thk =  2
cyl_od =  84
disc_dia = 79.5
disc_thk = 14
thru_hole = 16
counterbore = 3
groove_dia = 70
groove_height = 7
head_dia = 20
head_length = 7
neck_dia = 10
neck_length = 8
chamf_dist = 8.4
piston_dia = 20
piston_length = 168.5
ext_length_A = 25
ext_dia_A = 16
threaded_depth = 22.7
flange_dia = 96
flange_thk = 14
hub_od = 76
hub_id = 66
hub_length = 4
ext_dia_B = 80
ext_length_B = 4
center_hole_dia = 24
center_hole_depth = 9
bolt_hole_radius = 3
pattern_radius = 48
small_radius = 8
n_bolts = 3

# piston disc
cbore_height = 3
piston_disc = (cq.Workplane("XY").circle(disc_dia/2).extrude(disc_thk))
piston_disc = (piston_disc.faces(">Z").workplane()
               .cboreHole(thru_hole, counterbore, cbore_heigh

In [ ]:
#generate a piston disc for a pneumatic cylinder of ID 80 mm, stroke length 120 mm, and pressure of 6 bar

In [41]:
# Complete CAD-LLM Code Generator - Single Cell Solution
import sys
import re
import textwrap
import os
import builtins
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# For headless environments
if 'DISPLAY' not in os.environ:
    os.environ['DISPLAY'] = ':99'

try:
    import cadquery as cq
except ImportError as e:
    print("❌ CadQuery import failed. Please install system dependencies:")
    print("sudo apt-get install -y libgl1-mesa-glx libgl1-mesa-dev libegl1-mesa")
    print("pip install cadquery")
    sys.exit(1)

# Configuration
MODEL_DIR = "/home/ubuntu/cad-llm/cad_llm"
MAX_NEW_TOKENS = 512
MAX_INPUT_LEN = 256

def get_safe_builtins():
    """Return a restricted set of Python builtins (safe for CadQuery)."""
    allowed = {
        "abs", "min", "max", "round", "sum", "all", "any",
        "int", "float", "len", "range", "enumerate", "zip",
        "str", "bool", "list", "tuple", "dict", "set",
        "__import__"
    }
    return {k: getattr(builtins, k) for k in allowed}

def load_model(model_dir=MODEL_DIR):
    """Load the trained CAD-LLM model and tokenizer."""
    try:
        print(f"Loading model from {model_dir}...")
        tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        if torch.cuda.is_available():
            model = AutoModelForCausalLM.from_pretrained(
                model_dir, 
                device_map="auto",
                torch_dtype=torch.float16
            )
        else:
            model = AutoModelForCausalLM.from_pretrained(model_dir)
            model = model.to("cpu")
        
        model.eval()
        print("✅ Model loaded successfully")
        return tokenizer, model
    except Exception as e:
        print(f"❌ Failed to load model from {model_dir}: {e}")
        raise

def generate_code(prompt: str, tokenizer, model):
    """Generate CadQuery code from natural language prompt."""
    prompt_text = prompt.strip() + "\n### CADQUERY CODE\n"
    
    try:
        inputs = tokenizer(
            prompt_text, 
            return_tensors="pt", 
            truncation=True, 
            max_length=MAX_INPUT_LEN,
            padding=True
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
    except Exception as e:
        raise RuntimeError(f"Tokenization failed: {str(e)}")

    with torch.no_grad():
        try:
            gen = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                num_beams=4,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                repetition_penalty=1.1,
                early_stopping=True
            )
        except Exception as e:
            raise RuntimeError(f"Model generation failed: {str(e)}")

    if gen is None or len(gen) == 0:
        raise RuntimeError("Model returned empty generation")

    try:
        input_length = inputs['input_ids'].shape[1]
        output_seq = gen[0][input_length:]
        out = tokenizer.decode(output_seq, skip_special_tokens=True)
    except Exception as e:
        raise RuntimeError(f"Decoding failed: {str(e)}")

    if not out.strip():
        raise RuntimeError("Decoded output is empty")

    # Extract code after the separator
    if "### CADQUERY CODE" in out:
        parts = out.split("### CADQUERY CODE", 1)
        code = parts[1].strip() if len(parts) > 1 else out
    else:
        m = re.search(r"(import\s+cadquery\b.*)", out, re.S | re.I)
        code = m.group(1).strip() if m else out

    if not code.strip():
        raise RuntimeError("No code extracted from generation")
    
    return code

def clean_generated_code(code: str) -> str:
    """Clean and format the generated CadQuery code."""
    # Strip markdown fences
    code = re.sub(r"```[a-zA-Z]*", "", code).strip("` \n")

    # Normalize imports
    code = re.sub(r"\s*(import\s+cadquery)", r"\n\1", code, flags=re.IGNORECASE)
    
    # Normalize variable assignments
    code = re.sub(r"\s*(yield_strength|fos|P|cyl_id|cyl_len|cyl_thk|cyl_od|model|result|part)\s*=",
                  r"\n\1 = ", code)

    # Remove excessive trailing parentheses
    lines = code.splitlines()
    cleaned_lines = []
    paren_balance = 0
    
    for line in lines:
        if line.strip() == ")":
            if paren_balance <= 0:
                continue
        paren_balance += line.count("(") - line.count(")")
        cleaned_lines.append(line)

    code = "\n".join(cleaned_lines)

    # Balance parentheses if still mismatched
    open_parens = code.count('(')
    close_parens = code.count(')')
    if open_parens > close_parens:
        code += '\n' + (')' * (open_parens - close_parens))

    return textwrap.dedent(code).strip()

def fix_syntax_errors(code: str) -> str:
    """Fix common syntax errors in generated code."""
    lines = code.splitlines()
    fixed_lines = []
    
    for line in lines:
        # Fix mismatched parentheses in problematic lines
        if "groove_height - groove_height)/2)" in line:
            continue
        elif line.strip().endswith("/2)"):
            continue
        else:
            fixed_lines.append(line)
    
    return "\n".join(fixed_lines)

def complete_incomplete_code(code: str) -> str:
    """Try to complete incomplete generated code."""
    lines = code.splitlines()
    cleaned_lines = []
    
    for i, line in enumerate(lines):
        if line.strip().endswith(('.', '(', '=', '+', '-', '*', '/')):
            if i == len(lines) - 1 or (i < len(lines) - 1 and not lines[i + 1].strip()):
                continue
        cleaned_lines.append(line)
    
    code = "\n".join(cleaned_lines)
    
    # Find the main object variable name
    main_object = None
    for line in lines:
        if re.search(r"(piston_disc|cylinder|model|part|result)\s*=", line):
            match = re.search(r"(\w+)\s*=", line)
            if match:
                main_object = match.group(1)
                break
    
    # Add final assignment if missing
    if main_object and not re.search(r"\b(model|result|part)\s*=", code):
        code += f"\nmodel = {main_object}"
    
    return code

def simple_sanity_check(code_text: str):
    """Perform basic security and syntax checks on generated code."""
    forbidden = [
        "__import__", "import os", "import sys", "import subprocess",
        "open(", "eval(", "exec(", "compile(",
        "subprocess", "socket", "requests", "urllib", 
        "os.", "sys.", "globals", "locals", "__"
    ]
    
    lower = code_text.lower()
    for f in forbidden:
        if f in lower:
            raise RuntimeError(f"Forbidden pattern found in generated code: {f}")

    if "import cadquery" not in lower:
        raise RuntimeError("Generated code missing CadQuery import")
    
    if not re.search(r"\b(model|result|part)\s*=", code_text):
        raise RuntimeError("Generated code missing model/result/part assignment")
    
    return True

def exec_and_export(code_text: str, out_file="generated_from_llm.step"):
    """Execute the generated code and export to STEP file."""
    safe_globals = {"cq": cq, "__builtins__": get_safe_builtins()}
    safe_locals = {}

    try:
        exec(code_text, safe_globals, safe_locals)
    except Exception as e:
        raise RuntimeError(f"Error during code execution: {e}")

    # Find the CAD object
    cad_obj = None
    for name in ("model", "result", "part"):
        if name in safe_locals:
            cad_obj = safe_locals[name]
            break
        if name in safe_globals:
            cad_obj = safe_globals[name]
            break

    if cad_obj is None:
        raise RuntimeError("No CAD object found. Expected 'model'/'result'/'part' variable.")

    # Export to STEP file
    try:
        cq.exporters.export(cad_obj, out_file)
    except Exception as e:
        raise RuntimeError(f"Failed to export STEP file: {e}")
    
    return out_file

def generate_cad_from_prompt(prompt: str):
    """Main function to generate CAD from natural language prompt."""
    print("🔧 CAD-LLM Code Generator")
    print("=" * 50)

    if not prompt.strip():
        raise ValueError("Please provide a valid prompt.")

    print(f"🤖 Processing prompt: '{prompt}'")
    
    # Load model
    tokenizer, model = load_model(MODEL_DIR)

    try:
        print("🔄 Generating CadQuery code...")
        raw_code = generate_code(prompt, tokenizer, model)

        print("\n--- Raw generated code (first 500 chars) ---")
        print(raw_code[:500] + ("..." if len(raw_code) > 500 else ""))

        print("\n🧹 Cleaning generated code...")
        code = clean_generated_code(raw_code)
        
        print("\n🔧 Fixing syntax errors...")
        code = fix_syntax_errors(code)
        
        print("\n🔧 Completing incomplete code...")
        code = complete_incomplete_code(code)
        
        print("\n--- Final cleaned code ---")
        print(code)

        print("\n🔍 Performing sanity checks...")
        simple_sanity_check(code)

        print("\n⚙️ Executing code and exporting STEP file...")
        out_file = exec_and_export(code)
        print(f"\n✅ SUCCESS! STEP file created: {out_file}")
        print(f"📁 File size: {os.path.getsize(out_file)} bytes")
        
        return out_file, code

    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        # Save debug info
        try:
            debug_file = "last_raw_code.txt"
            with open(debug_file, "w") as f:
                f.write(raw_code if 'raw_code' in locals() else "No code generated")
            print(f"🐛 Debug info saved to {debug_file}")
        except:
            pass
        raise

# ========== MAIN EXECUTION ==========
print("🚀 CAD-LLM Complete Generator Ready!")
print("=" * 60)

# Get user input
prompt = input("Enter your CAD description: ").strip()

if not prompt:
    print("❌ Please provide a valid prompt.")
else:
    try:
        out_file, code = generate_cad_from_prompt(prompt)
        print(f"\n🎉 Generated CAD successfully: {out_file}")
        print(f"📂 File location: {os.path.abspath(out_file)}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        import traceback
        traceback.print_exc()


🚀 CAD-LLM Complete Generator Ready!


🔧 CAD-LLM Code Generator
🤖 Processing prompt: 'generate a piston disc for a pneumatic cylinder of ID 80 mm, stroke length 120 mm, and pressure of 6 bar'
Loading model from /home/ubuntu/cad-llm/cad_llm...
✅ Model loaded successfully
🔄 Generating CadQuery code...

--- Raw generated code (first 500 chars) ---
import cadquery as cq

# Parameters from dataset row
pressure_bar = 6.5
pressure_mpa = 0.65
stroke_length = 120
cyl_id = 80
cyl_len = 150
cyl_thk = 2
cyl_od = 84
disc_dia = 79.5
disc_thk = 14
thru_hole = 16
counterbore = 3
groove_dia = 70
groove_height = 7
head_dia = 20
head_length = 7
neck_dia = 10
neck_length = 8
chamf_dist = 8.4
piston_dia = 20
piston_length = 168.5
ext_length_A = 25
ext_dia_A = 16
threaded_depth = 22.7
flange_dia = 96
flange_thk = 14
hub_od = 76
hub_id = 66
hub_length = 4
ex...

🧹 Cleaning generated code...

🔧 Fixing syntax errors...

🔧 Completing incomplete code...

--- Final cleaned code ---
import cadquery as cq

# Parameters from dataset row
pressure_bar = 6.

Traceback (most recent call last):
  File "/tmp/ipykernel_16922/3015484819.py", line 227, in exec_and_export
    exec(code_text, safe_globals, safe_locals)
  File "<string>", line 46
    .circle(groove_dia/2).circle(disc_dia/2).cutBlind(-groove_height))
IndentationError: unexpected indent

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/tmp/ipykernel_16922/3015484819.py", line 317, in <module>
    out_file, code = generate_cad_from_prompt(prompt)
  File "/tmp/ipykernel_16922/3015484819.py", line 288, in generate_cad_from_prompt
    out_file = exec_and_export(code)
  File "/tmp/ipykernel_16922/3015484819.py", line 229, in exec_and_export
    raise RuntimeError(f"Error during code execution: {e}")
RuntimeError: Error during code execution: unexpected indent (<string>, line 46)


In [54]:
# Complete CAD-LLM Code Generator - Single Cell Solution (fixed)
import sys, re, textwrap, os, builtins, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Headless hint (CadQuery often needs a DISPLAY even if unused)
os.environ.setdefault("DISPLAY", ":99")

# --- CadQuery import ---
try:
    import cadquery as cq
except Exception as e:
    print("❌ CadQuery import failed. You may need system deps and cadquery:")
    print("   sudo apt-get install -y libgl1-mesa-glx libgl1-mesa-dev libegl1-mesa")
    print("   pip install cadquery")
    raise

# --- Config ---
MODEL_DIR = os.environ.get("CAD_LLM_DIR", "/home/ubuntu/cad-llm/cad_llm")
MAX_NEW_TOKENS = 512
MAX_INPUT_LEN = 256
SEPARATOR = "\n### CADQUERY CODE\n"

# Force the model down the piston-disc route by seeding the code:
PISTON_DISC_PREFIX = """import cadquery as cq

# piston disc
"""

# Stop when we see a show_object call (any variable)
STOP_REGEX = re.compile(r"show_object\s*\(")

# ---------- Safety ----------
def get_safe_builtins():
    """Restricted Python builtins for exec sandbox."""
    allowed = {
        "abs","min","max","round","sum","all","any",
        "int","float","len","range","enumerate","zip",
        "str","bool","list","tuple","dict","set",
        "__import__",  # required for 'import cadquery' to work inside exec
    }
    return {k: getattr(builtins, k) for k in allowed}

FORBIDDEN_SNIPPETS = [
    "import os", "import sys", "import subprocess",
    "subprocess", "socket", "requests", "urllib", "shutil",
    "eval(", "exec(", "compile(", "open(", "os.", "sys.",
    "__import__(",  # explicit use in text (Python import still works via builtins)
]

# ---------- Model load ----------
def load_model(model_dir=MODEL_DIR):
    print(f"Loading model from {model_dir}...")
    tok = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    if torch.cuda.is_available():
        model = AutoModelForCausalLM.from_pretrained(
            model_dir, device_map="auto", torch_dtype=torch.float16
        )
    else:
        model = AutoModelForCausalLM.from_pretrained(model_dir).to("cpu")

    # 🔑 Ensure embeddings match tokenizer size
    model.resize_token_embeddings(len(tok))

    model.eval()
    print("✅ Model loaded")
    return tok, model


# ---------- Generation ----------
def _strip_to_piston_disc(code: str) -> str:
    """
    If the model rambles into other parts (piston_rod, flange),
    keep only the piston disc section.
    """
    # Prefer the region starting at "# piston disc" if present
    m = re.search(r"(?ms)^#\s*piston\s*disc\b.*", code)
    if m:
        code = m.group(0)

    # Hard block: drop any 'piston_rod' or 'flange' blocks that follow
    code = re.split(r"(?ms)^\s*#\s*piston\s*rod\b|^\s*#\s*flange\b", code)[0]
    return code.strip()


def generate_code(prompt: str, tokenizer, model,
                  max_rounds=4, max_new_tokens=400, min_new_tokens=120):
    """
    Iteratively generate until we hit 'show_object(' or rounds exhausted.
    Trims context to model.config.n_positions (GPT-2 = 1024) each round.
    """
    prefix = prompt.strip() + SEPARATOR
    inputs = tokenizer(
        prefix, return_tensors="pt",
        truncation=True, max_length=MAX_INPUT_LEN, padding=True
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    context_ids = inputs["input_ids"]
    attn = inputs["attention_mask"]
    max_ctx = getattr(model.config, "n_positions", 1024)

    collected = ""
    for round_idx in range(max_rounds):
        # 🔑 Trim to max context length
        if context_ids.shape[1] > max_ctx:
            context_ids = context_ids[:, -max_ctx:]
            attn = attn[:, -max_ctx:]

        with torch.no_grad():
            out = model.generate(
                input_ids=context_ids,
                attention_mask=attn,
                max_new_tokens=max_new_tokens,
                min_new_tokens=min_new_tokens,
                do_sample=True,
                temperature=0.18,
                top_p=0.95,
                top_k=50,
                repetition_penalty=1.05,
                num_beams=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                use_cache=True,
            )

        # Take only the newly generated tokens
        new_tokens = out[0, context_ids.shape[1]:]
        new_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
        collected += new_text

        # Update context for next round
        context_ids = out
        attn = torch.ones_like(context_ids, device=context_ids.device)

        # Stop condition
        if "show_object(" in collected:
            break

    code = collected.strip()
    if not code:
        raise RuntimeError("No code extracted from generation")
    return code



# ---------- Cleaning / Repair ----------
def ensure_header(code: str) -> str:
    """Ensure the code starts with 'import cadquery as cq' or provide cq in scope."""
    if re.search(r"\bimport\s+cadquery\b.*\bas\b.*\bcq\b", code, re.I):
        return code
    # If it already has 'import cadquery' without alias, add alias line
    if re.search(r"\bimport\s+cadquery\b", code, re.I):
        return "import cadquery as cq\n" + code
    # If no import at all, inject the import so code can run
    return "import cadquery as cq\n" + code

def clean_generated_code(code: str) -> str:
    # strip backtick fences
    code = re.sub(r"```[a-zA-Z]*", "", code).strip("`\n ")

    # normalize 'import cadquery' to a clean top line
    code = ensure_header(code)

    # normalize some assignments with spacing
    code = re.sub(
        r"\s*(yield_strength|fos|P|cyl_id|cyl_len|cyl_thk|cyl_od|model|result|part)\s*=",
        r"\n\1 = ",
        code,
    )

    # prune lone closing parens at start of lines when unbalanced
    lines = code.splitlines()
    cleaned, bal = [], 0
    for ln in lines:
        if ln.strip() == ")" and bal <= 0:
            continue
        bal += ln.count("(") - ln.count(")")
        cleaned.append(ln)
    code = "\n".join(cleaned)

    # close any remaining opens
    opens = code.count("(")
    closes = code.count(")")
    if opens > closes:
        code += "\n" + (")" * (opens - closes))

    return textwrap.dedent(code).strip()

def _tidy_method_chains(code: str) -> str:
    """
    Fix common chain formatting issues:
    - Remove lines that are only '.' or ',' (stray punctuation).
    - If a line is just '.' (or starts with '.'), glue it to previous non-empty line.
    - Normalize indentation of chained calls.
    """
    lines = code.splitlines()
    out = []
    for i, ln in enumerate(lines):
        raw = ln.rstrip()

        # Drop pure stray punctuation lines
        if raw.strip() in {".", ","}:
            continue

        # If a line starts with a single dot like ".circle(...)" but previous line
        # does not end with an operator, prepend a safe continuation
        if raw.lstrip().startswith(".") and out:
            prev = out[-1].rstrip()
            # If previous already ends with a closing paren, it's fine
            # otherwise append a backslash to explicitly continue (legal but optional)
            out[-1] = prev  # keep as-is
            out.append(raw.lstrip())
            continue

        out.append(raw)
    return "\n".join(out)


def fix_syntax_errors(code: str) -> str:
    """
    Fix common syntax glitches from the generator:
    - Remove known bad fragments.
    - Tidy method chains & stray punctuation lines.
    - Ensure balanced parentheses (light pass; heavy balance is done in clean_generated_code).
    """
    # Remove specific half-emitted fragments if they appear
    bad_fragments = [
        "groove_height - groove_height)/2)",   # observed garble
    ]
    for frag in bad_fragments:
        code = "\n".join([ln for ln in code.splitlines() if frag not in ln])

    # Tidy method chains & drop '.' lines
    code = _tidy_method_chains(code)

    # If a line ends with just a dot attached to a closing paren, drop the dot
    code = re.sub(r"\)\s*\.\s*(\n|$)", r")\n", code)

    # Remove any trailing solitary '.' at EOF
    code = re.sub(r"\.\s*$", "", code)

    return code


def ensure_model_assignment(code: str) -> str:
    """
    Ensure there is a 'model = <shape>' assignment.
    If missing, try to use the last assigned variable as model.
    """
    if re.search(r"\b(model|result|part)\s*=", code):
        return code

    # Prefer variables that look like CAD objects (appear on the LHS)
    candidates = []
    for ln in code.splitlines():
        m = re.match(r"\s*([A-Za-z_][A-Za-z0-9_]*)\s*=\s*", ln)
        if m:
            candidates.append(m.group(1))

    if candidates:
        code += f"\nmodel = {candidates[-1]}"
    else:
        # As a last resort, create an empty workplane so export doesn't crash
        code += "\nmodel = cq.Workplane('XY')"
    return code

    # Heuristic: last variable assigned to a CQ operation
    candidates = []
    for ln in code.splitlines():
        m = re.match(r"\s*([A-Za-z_][A-Za-z0-9_]*)\s*=\s*", ln)
        if m:
            name = m.group(1)
            if re.search(rf"\b{name}\s*=\s*.*cq\.", ln):
                candidates.append(name)
            else:
                candidates.append(name)
    main = candidates[-1] if candidates else None
    if main:
        code += f"\nmodel = {main}"
    return code

# ---------- Checks ----------
def simple_sanity_check(code_text: str):
    low = code_text.lower()
    for token in FORBIDDEN_SNIPPETS:
        if token in low:
            raise RuntimeError(f"Forbidden pattern in generated code: {token}")
    # not strictly required if we inject import, but still useful
    if "import cadquery" not in low:
        raise RuntimeError("Generated code missing 'import cadquery' import line")
    return True

# ---------- Exec & Export ----------
def exec_and_export(code_text: str, out_file="generated_from_llm.step"):
    safe_globals = {"cq": cq, "__builtins__": get_safe_builtins()}
    safe_locals = {}
    try:
        exec(code_text, safe_globals, safe_locals)
    except Exception as e:
        raise RuntimeError(f"Error during code execution: {e}")

    # Find the CAD object
    cad_obj = None
    for name in ("model", "result", "part"):
        if name in safe_locals:
            cad_obj = safe_locals[name]
            break
        if name in safe_globals:
            cad_obj = safe_globals[name]
            break
    if cad_obj is None:
        raise RuntimeError("No CAD object found. Expected variable: model/result/part")

    try:
        cq.exporters.export(cad_obj, out_file)
    except Exception as e:
        raise RuntimeError(f"Failed to export STEP file: {e}")
    return out_file

# ---------- Orchestration ----------
def generate_cad_from_prompt(prompt: str):
    print("🔧 CAD-LLM Code Generator")
    print("=" * 50)
    if not prompt.strip():
        raise ValueError("Please provide a valid prompt.")

    print(f"🤖 Prompt: {prompt!r}")
    tokenizer, model = load_model(MODEL_DIR)

    try:
        print("🔄 Generating CadQuery code...")
        raw = generate_code(prompt, tokenizer, model)
        print("\n--- Raw generation (first 500 chars) ---")
        print(raw[:500] + ("..." if len(raw) > 500 else ""))

        print("\n🧹 Cleaning...")
        code = clean_generated_code(raw)

        print("🔧 Fixing syntax...")
        code = fix_syntax_errors(code)

        print("🧩 Ensuring model assignment...")
        code = ensure_model_assignment(code)

        print("\n--- Final cleaned code ---")
        print(code)

        print("\n🔍 Sanity checks...")
        simple_sanity_check(code)

        print("⚙️ Executing and exporting STEP...")
        out_file = exec_and_export(code)
        print(f"✅ STEP file created: {out_file}  (size: {os.path.getsize(out_file)} bytes)")
        return out_file, code

    except Exception as e:
        print(f"\n❌ ERROR: {e}")
        # Save for debugging
        try:
            with open("last_raw_code.txt", "w") as f:
                f.write(raw if 'raw' in locals() else "[no raw code captured]")
            print("🐛 Saved raw generation to last_raw_code.txt")
        except Exception:
            pass
        raise

# ========== MAIN ==========
print("🚀 CAD-LLM Complete Generator Ready!")
print("=" * 60)

try:
    prompt = input("Enter your CAD description: ").strip()
except EOFError:
    # Non-interactive fallback (e.g., piped execution)
    prompt = ""

if not prompt:
    print("❌ Please provide a valid prompt.")
else:
    try:
        out_file, code = generate_cad_from_prompt(prompt)
        print(f"\n🎉 Generated CAD successfully: {out_file}")
        print(f"📂 File location: {os.path.abspath(out_file)}")
    except Exception as e:
        print(f"❌ Failed: {e}")
        import traceback; traceback.print_exc()


🚀 CAD-LLM Complete Generator Ready!


🔧 CAD-LLM Code Generator
🤖 Prompt: 'generate a piston rod for a pneumatic cylinder of ID 120 mm, stroke length 150 mm, and a pressure of 6 bar'
Loading model from /home/ubuntu/cad-llm/cad_llm...
✅ Model loaded
🔄 Generating CadQuery code...

❌ ERROR: index out of range in self
🐛 Saved raw generation to last_raw_code.txt
❌ Failed: index out of range in self


Traceback (most recent call last):
  File "/tmp/ipykernel_16922/3124131107.py", line 384, in <module>
    out_file, code = generate_cad_from_prompt(prompt)
  File "/tmp/ipykernel_16922/3124131107.py", line 335, in generate_cad_from_prompt
    raw = generate_code(prompt, tokenizer, model)
  File "/tmp/ipykernel_16922/3124131107.py", line 113, in generate_code
    out = model.generate(
  File "/home/ubuntu/cad-llm/.venv/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 120, in decorate_context
    return func(*args, **kwargs)
  File "/home/ubuntu/cad-llm/.venv/lib/python3.10/site-packages/transformers/generation/utils.py", line 2539, in generate
    result = self._sample(
  File "/home/ubuntu/cad-llm/.venv/lib/python3.10/site-packages/transformers/generation/utils.py", line 2870, in _sample
    outputs = model_forward(**model_inputs, return_dict=True)
  File "/home/ubuntu/cad-llm/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1773, in _wrapped_call_impl